In [1]:
import streamlit as st
import pandas as pd
import mysql.connector
from datetime import datetime

In [41]:
conn = mysql.connector.connect(
    host="localhost",
    user="root",
    password="",  
    database="traffic_data"  
)
cursor = conn.cursor()

In [43]:
cursor.execute("""
CREATE TABLE IF NOT EXISTS check_post_logs (
    id INT AUTO_INCREMENT PRIMARY KEY,
    vehicle_number VARCHAR(20),
    driver_id VARCHAR(100),
    officer_id VARCHAR(50),
    timestamp DATETIME,
    status VARCHAR(50)
)
""")

In [45]:
csv_path = "traffic.csv"
df = pd.read_csv(csv_path)
df.dropna(axis=1, how='all', inplace=True)
for column in df.columns:
    if df[column].dtype == 'object':
        df[column].fillna("Unknown", inplace=True)
    else:
        df[column].fillna(0, inplace=True)

df["vehicle_number"] = ["TN12AB" + str(1000 + i) for i in range(len(df))]
df["officer_id"] = ["O" + str(100 + (i % 5)) for i in range(len(df))]
df["driver_id"] = df["driver_gender"] + "_" + df["driver_age"].astype(str)
df["timestamp"] = pd.to_datetime(df["stop_date"] + ' ' + df["stop_time"], errors='coerce')
df["status"] = df["stop_outcome"]

C:\Users\Dell\AppData\Local\Temp\ipykernel_7312\3163283369.py:2: DtypeWarning: Columns (10) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(csv_path)
C:\Users\Dell\AppData\Local\Temp\ipykernel_7312\3163283369.py:6: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df[column].fillna("Unknown", inplace=True)
C:\Users\Dell\AppData\Local\Temp\ipykernel_7312\3163283369.py:8: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inpl

In [23]:
for _, row in df.iterrows():
    try:
        cursor.execute("""
        INSERT INTO check_post_logs (vehicle_number, driver_id, officer_id, timestamp, status)
        VALUES (%s, %s, %s, %s, %s)
        """, (
            row["vehicle_number"],
            row["driver_id"],
            row["officer_id"],
            row["timestamp"].strftime('%Y-%m-%d %H:%M:%S') if pd.notnull(row["timestamp"]) else None,
            row["status"]
        ))
    except Exception as e:
        print("Insert error:", e)
conn.commit()

In [24]:
cursor.execute("SELECT * FROM check_post_logs")
logs = pd.DataFrame(cursor.fetchall(), columns=[i[0] for i in cursor.description])
logs.head(50)

,id,vehicle_number,driver_id,officer_id,timestamp,status
0,1,TN12AB1000,M_19,O100,2020-01-01 00:00:00,Ticket
1,2,TN12AB1001,M_58,O101,2020-01-01 00:01:00,Arrest
2,3,TN12AB1002,M_76,O102,2020-01-01 00:02:00,Ticket
3,4,TN12AB1003,M_76,O103,2020-01-01 00:03:00,Warning
4,5,TN12AB1004,M_75,O104,2020-01-01 00:04:00,Arrest
5,6,TN12AB1005,F_73,O100,2020-01-01 00:05:00,Ticket
6,7,TN12AB1006,F_70,O101,2020-01-01 00:06:00,Ticket
7,8,TN12AB1007,M_50,O102,2020-01-01 00:07:00,Arrest
8,9,TN12AB1008,F_30,O103,2020-01-01 00:08:00,Arrest
9,10,TN12AB1009,M_21,O104,2020-01-01 00:09:00,Warning


In [27]:
search = "TN12AB1011"
cursor.execute("SELECT * FROM check_post_logs WHERE vehicle_number = %s", (search,))
result = pd.DataFrame(cursor.fetchall(), columns=[i[0] for i in cursor.description])
result

,id,vehicle_number,driver_id,officer_id,timestamp,status
0,12,TN12AB1011,F_53,O101,2020-01-01 00:11:00,Arrest


In [29]:
cursor.execute("SELECT officer_id, COUNT(*) AS stop_count FROM check_post_logs GROUP BY officer_id")
pd.DataFrame(cursor.fetchall(), columns=[i[0] for i in cursor.description])

,officer_id,stop_count
0,O100,13108
1,O101,13108
2,O102,13108
3,O103,13107
4,O104,13107


In [31]:
cursor.close()
conn.close()

In [33]:
!pip install tabulate

Defaulting to user installation because normal site-packages is not writeable


In [35]:
from tabulate import tabulate